In [1]:
import sys
import os

__file__ = 'helper_funcs.py'
file_path = os.path.abspath(os.path.join(os.path.dirname( __file__ ), '..'))
sys.path.append(file_path)

import helper_funcs as hf


In [142]:
# raw_input = '''...#......
# .......#..
# #.........
# ..........
# ......#...
# .#........
# .........#
# ..........
# .......#..
# #...#.....'''

raw_input = hf.read_file('input.txt')

mod_input = hf.modify_raw(raw_input)

def find_expanse(input:list)->list:
    #horizontal
    vert = []
    hor = []
    for n,i in enumerate(input): 
        if all(v=='.' for v in i):
            hor.append(n)
    #vertical
    for i in range(len(input[0])):
        tmp = []
        for j in input:
            tmp.append(j[i])
        
        if all(q=='.' for q in tmp):
            vert.append(i)

    return [hor,vert]


def adjust_map(input:list)->list:
    expanses = find_expanse(mod_input)
    copy = mod_input.copy()
    #horizontal
    gen_template = ''.join(['.' for i in range(len(input[0]))])
    t = 0
    for i in expanses[0]:
        copy.insert(i+t,gen_template)
        t+=1
    
    #vert
    t = 0
    for j in expanses[1]: #2 5 8
        for z in range(len(copy)):  #line by line number
            tmp = list(copy[z])
            tmp.insert(j+t,'.')
            copy[z] = ''.join(tmp)
        t+=1

    return copy


def find_galaxies(map:list)->list:
    galaxy_coors = []
    for i in range(len(map[0])):
        for j in range(len(map)):
            coor = (i,j)
            if map[j][i] == '#':
                galaxy_coors.append(coor)
    return galaxy_coors


new_map = adjust_map(mod_input)
find_galaxies(new_map)

[(0, 15),
 (0, 25),
 (0, 42),
 (1, 77),
 (1, 113),
 (1, 145),
 (2, 11),
 (2, 84),
 (3, 60),
 (3, 94),
 (3, 107),
 (3, 126),
 (3, 139),
 (3, 149),
 (4, 1),
 (4, 17),
 (4, 68),
 (4, 100),
 (4, 121),
 (5, 51),
 (5, 131),
 (6, 81),
 (6, 117),
 (7, 9),
 (7, 32),
 (7, 46),
 (7, 72),
 (7, 110),
 (8, 15),
 (8, 140),
 (9, 26),
 (9, 41),
 (9, 67),
 (9, 102),
 (9, 126),
 (10, 1),
 (10, 54),
 (10, 61),
 (10, 149),
 (11, 84),
 (12, 90),
 (13, 24),
 (13, 35),
 (13, 44),
 (13, 105),
 (13, 116),
 (13, 146),
 (14, 12),
 (14, 73),
 (14, 141),
 (15, 5),
 (15, 30),
 (15, 51),
 (15, 126),
 (16, 100),
 (17, 22),
 (17, 63),
 (17, 84),
 (17, 110),
 (17, 136),
 (18, 43),
 (18, 90),
 (18, 145),
 (19, 36),
 (19, 69),
 (19, 104),
 (19, 118),
 (19, 131),
 (20, 140),
 (21, 19),
 (21, 27),
 (22, 33),
 (22, 81),
 (22, 101),
 (22, 135),
 (23, 46),
 (23, 87),
 (24, 10),
 (24, 38),
 (24, 56),
 (24, 77),
 (24, 105),
 (24, 124),
 (25, 147),
 (26, 43),
 (27, 32),
 (27, 59),
 (27, 73),
 (28, 25),
 (28, 66),
 (28, 84),
 (28,

In [67]:
def distance(c1:tuple,c2:tuple):
    tot = 0
    if len(c1) == len(c2):
        for i in range(len(c1)):
            tot+= (c2[i]-c1[i])**2
    return tot**(1/2)

def ceil(a):
    return a//1 + bool(a%1)

ceil(distance((1,6),(5,11)))

7.0

In [143]:
def get_dim(matrix:list) -> tuple:
    #returns the dimensions of a matrix
    return (len(matrix[0]),len(matrix))

def clamp(matrix:list,coord:tuple)->bool:
    #mchecks the coordinate to make sure it's within the dim of the matrix, returns false of it's outside
    dim = get_dim(matrix)
    if (coord[0]< 0) or (coord[0]>dim[0]-1) or (coord[1]<0) or (coord[1]>dim[1]-1):
        return False
    else:
        return True

def neighbors(coor:tuple) -> list:
    #finds the neighbors of a cell including diagonal
    coords = []
    for i in [-1,0,1]:
        for j in [-1,0,1]:
            if (abs(i)==abs(j)): #omit corners, make sure it's only 1 moved and the other constant
                pass
            else:
                coords.append((coor[0]+i,coor[1]+j))
    return coords

def find_cell(matrix:list,coor:tuple) -> str:
    #finds the value of a cell
    x = coor[0]
    y = coor[1]
    return matrix[y][x]

In [149]:
#a* implementation

def astar(start:tuple,end:tuple):
    graph = []
    for j in range(0,max(start[1],end[1])+1):
        leaf = []
        for i in range(0,max(start[0],end[0])+1):
            h = distance((i,j),end) #heuristic
            
            leaf.append(h)
        graph.append(leaf)

    min_energy = find_cell(graph,start)
    next_cell = start
    l = 0
    tmp_map = new_map.copy()  ##uh oh global variable wee woo
    while min_energy != 0:
        for i in neighbors(next_cell):
            if clamp(graph,i):
                cell_energy = find_cell(graph,i)
                if cell_energy < min_energy:
                    min_energy = cell_energy
                    next_cell = i
       
        x = next_cell[0]
        y = next_cell[1]
        temp_list = list(tmp_map[y])
        temp_list[x] = 'x'
        tmp_map[y] = ''.join(temp_list)
        
     
        l +=1
    # print('\n'.join(tmp_map))

    return l

###########

new_map = adjust_map(mod_input)
galaxies = find_galaxies(new_map)
tracker = []

lengths = {}
print(len(galaxies))
for i in galaxies:
    for j in galaxies:
        if i != j:
            v = astar(i,j)
            lengths.setdefault(''.join(sorted([str(i),str(j)])),v)
    tracker.append(i)
    print(f'{len(tracker)/len(galaxies)*100:.2f}%')
sum(lengths.values())


440
0.23%
0.45%
0.68%
0.91%
1.14%
1.36%
1.59%
1.82%
2.05%
2.27%
2.50%
2.73%
2.95%
3.18%
3.41%
3.64%
3.86%
4.09%
4.32%
4.55%
4.77%
5.00%
5.23%
5.45%
5.68%
5.91%
6.14%
6.36%
6.59%
6.82%
7.05%
7.27%
7.50%
7.73%
7.95%
8.18%
8.41%
8.64%
8.86%
9.09%
9.32%
9.55%
9.77%
10.00%
10.23%
10.45%
10.68%
10.91%
11.14%
11.36%
11.59%
11.82%
12.05%
12.27%
12.50%
12.73%
12.95%
13.18%
13.41%
13.64%
13.86%
14.09%
14.32%
14.55%
14.77%
15.00%
15.23%
15.45%
15.68%
15.91%
16.14%
16.36%
16.59%
16.82%
17.05%
17.27%
17.50%
17.73%
17.95%
18.18%
18.41%
18.64%
18.86%
19.09%
19.32%
19.55%
19.77%
20.00%
20.23%
20.45%
20.68%
20.91%
21.14%
21.36%
21.59%
21.82%
22.05%
22.27%
22.50%
22.73%
22.95%
23.18%
23.41%
23.64%
23.86%
24.09%
24.32%
24.55%
24.77%
25.00%
25.23%
25.45%
25.68%
25.91%
26.14%
26.36%
26.59%
26.82%
27.05%
27.27%
27.50%
27.73%
27.95%
28.18%
28.41%
28.64%
28.86%
29.09%
29.32%
29.55%
29.77%
30.00%
30.23%
30.45%
30.68%
30.91%
31.14%
31.36%
31.59%
31.82%
32.05%
32.27%
32.50%
32.73%
32.95%
33.18%
33.41%
33.64%
33.

9627977